In [2]:
import pandas as pd 
import numpy as np 
df1=pd.read_csv('./lingo_ex_movie.csv')
df2=pd.read_csv('./lingo_ex_rating.csv')
df1.head(3)

,Unnamed: 0,belongs_to_collection,genres,id,production_companies,release_date,title,vote_average,vote_count,series_num
0,0,Toy Story Collection,"['Animation', 'Comedy', 'Family']",862,Pixar Animation Studios,1995-10-30,Toy Story,7.7,5415.0,1
1,1,NaN,"['Adventure', 'Fantasy', 'Family']",8844,TriStar Pictures,1995-12-15,Jumanji,6.9,2413.0,0
2,2,Grumpy Old Men Collection,"['Romance', 'Comedy']",15602,Warner Bros.,1995-12-22,Grumpier Old Men,6.5,92.0,1


In [3]:
df2.head(3)

,userId,movieId,rating,date
0,1,110,1.0,2015-03-09
1,1,147,4.5,2015-03-09
2,1,858,5.0,2015-03-09


In [4]:
test = df2.groupby(['date']).size().reset_index(name='counts')
test = test.sort_values(by= ['counts'], ascending=False)
test.head(10)

,date,counts
1682,2000-11-20,97319
3264,2005-03-22,79428
1337,1999-12-11,68751
4579,2008-10-29,57010
1683,2000-11-21,56050
3265,2005-03-23,54897
1339,1999-12-13,52914
1338,1999-12-12,49795
1681,2000-11-19,43972
1340,1999-12-14,39686


In [5]:
print(test.loc[test['date'] == '2000-11-20'])
print(test.loc[test['date'] == '2000-11-21'])

            date  counts
1682  2000-11-20   97319
            date  counts
1683  2000-11-21   56050


In [6]:

one_day = df2.loc[df2['date'] == '2000-11-20'].groupby(['movieId']).agg({'rating': ['mean','count']})
next_day = df2.loc[df2['date'] == '2000-11-21'].groupby(['movieId']).agg({'rating': ['mean','count']})
one_day.columns = one_day.columns.droplevel(0)
next_day.columns = next_day.columns.droplevel(0)
one_day.head(3)

,mean,count
movieId,,
1,4.132479,234
2,3.029851,67
3,2.636364,66


In [7]:
C0= one_day['mean'].mean()
m0= one_day['count'].quantile(0.4)
C1= next_day['mean'].mean()
m1= next_day['count'].quantile(0.4)
print(C0,m0,C1,m1)

one_filter = one_day.loc[one_day['count'] >= m0]
next_filter = next_day.loc[next_day['count'] >= m1]

def weighted_rating(x, m, C):
    v = x['count']
    R = x['mean']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

one_filter['new_average'] = one_filter.apply(weighted_rating, axis=1, args=(m0,C0))
next_filter['new_average'] = next_filter.apply(weighted_rating, axis=1, args=(m1,C1))

3.2133333524881125 9.0 3.1885676578903883 6.0


<ipython-input-7-e5b6d5bb82be>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_filter['new_average'] = one_filter.apply(weighted_rating, axis=1, args=(m0,C0))
<ipython-input-7-e5b6d5bb82be>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  next_filter['new_average'] = next_filter.apply(weighted_rating, axis=1, args=(m1,C1))


In [59]:
#Sort movies based on score calculated above
one_filter = one_filter.sort_values('new_average', ascending=False)
next_filter = next_filter.sort_values('new_average', ascending=False)

one_filter.head(10)

,mean,count,new_average
movieId,,,
318,4.563380,213,4.508649
50,4.552486,181,4.489053
1148,4.574468,94,4.455534
527,4.463023,311,4.427875
858,4.457447,282,4.418969
908,4.471204,191,4.414600
1213,4.461140,193,4.405545
750,4.472973,148,4.400764
260,4.429775,356,4.399781


In [60]:
next_filter.head(10)

,mean,count,new_average
movieId,,,
750,4.671642,67,4.549745
1198,4.566434,143,4.510949
527,4.544118,136,4.486841
904,4.543860,57,4.414784
858,4.463415,123,4.404119
923,4.515625,64,4.401877
2019,4.633333,30,4.392539
913,4.508197,61,4.390021
922,4.606061,33,4.387985


In [58]:
index = one_filter.index
for i in range(10):
    print(df1[['id','title']].loc[df1['id'] == str(index[i])])

       id                     title
4020  318  The Million Dollar Hotel
Empty DataFrame
Columns: [id, title]
Index: []
Empty DataFrame
Columns: [id, title]
Index: []
      id               title
286  527  Once Were Warriors
      id                 title
534  858  Sleepless in Seattle
Empty DataFrame
Columns: [id, title]
Index: []
        id                    title
3059  1213  The Talented Mr. Ripley
       id            title
8546  750  Murder She Said
      id         title
938  260  The 39 Steps
       id        title
459  2019  Hard Target
